# Deployment of Lagos Water Availability Project

## Installing the libraries

In [ ]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 112 kB 44.4 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 84 kB 3.2 MB/s 
     |████████████████████████████████| 270 kB 68.5 MB/s 
     |████████████████████████████████| 2.3 MB 49.4 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 212 kB 51.6 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
     |████████████████████████████████| 80 kB 8.4 MB/s 
     |████████████████████████████████| 68 kB 6.8 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 593 kB 46.2 MB/s 
     |████████████████████████████████| 4.0 MB 34.

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 4.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 4.4 MB 41.5 MB/s 
     |████████████████████████████████| 260 kB 55.2 MB/s 
     |████████████████████████████████| 4.3 MB 38.3 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.15.0-py3-none-any.whl size=6641946 sha256=87077f5de8ee21b2f820524afecec0aa8d26d95678138d28a9a7d18470a01191
  Stored in directory: /root/.cache/pip/wheels/26/57/ce/ca63ad74b90273f9a682028d187645a42dce5c5255228d46c8
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=7a4cfdc81ca0844d50717aeb481d1a139538cbb8954562eecf166d195b5eab13
  Sto

**Restart Runtime from here before proceeding to import the libraries**

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import io
import json
import pickle
import gradio as gr
from sklearn.model_selection import train_test_split, StratifiedKFold
from autosklearn.classification import AutoSklearnClassifier
import matplotlib.pyplot as plt

### Reading the json and pickle file

In [ ]:
with open('/content/one_hot_cols.json') as json_file: 
   encoded_data = json.load(json_file)

In [ ]:
with open('/content/water auto-classifier.pkl', 'rb') as f:
    classifier = pickle.load(f)

### Defining functions used in Feature Engineering

In [ ]:
def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

In [ ]:
def lg(x):
  return np.log2(x)

In [ ]:
expected_columns = ['Classification as urban/semi-urban/rural_one-hot', 'log_Population',
       'water_source_category_one-hot', 'log_angle', '#status_id_one-hot',
       'log_days_passed', '#adm2_one-hot', 'log_dist', '#pay_one-hot',
       'water_tech_category_one-hot', 'Water demand (lpcd)_one-hot']

### Feature Engineering

In [ ]:
def feature_engineering():
  inp = []
  print( "Enter classification as of the area as \n 0. urban \n 1. semi-urban \n 2. rural: \n (Enter 0, 1, 2)")
  a1 = int(input())
  inp.append(a1)

  print( "Enter the population of the area: (Enter a positive integer)")
  a2 = int(input())
  inp.append(lg(a2)) #log population

  print( "Enter the water source category of the area as \n 0. well \n 1. spring \n (Enter 0, 1)")
  a3 = int(input())
  inp.append(a3)

  print( "Enter the latitude of the area:")
  a4 = int(input())
  print( "Enter the latitude of the area:")
  a5 = int(input())
  a4, a5 = cart2pol(a4, a5)

  inp.append(lg(a4 + 1000)) #log angle

  print( "Enter the status id of water source as \n 0. Yes (Functional) \n 1. No (non-functional): \n (Enter 0, 1)")
  a6 = int(input())
  inp.append(a6)

  print( "Enter the number of days passed since installation of water source: (Enter a postive integer)")
  a7 = int(input())
  inp.append(lg(a7))

  print( "Enter the local government name as")
  i = 0
  for adm in encoded_data['#adm2']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print('(Enter a number from 0 to ' + str(i) + ')')
  print( str(i) + ". " + 'Unknown' )
  a8 = int(input())
  inp.append(a8)

  inp.append(lg(a5 +1000)) #log of distance

  print( "Enter the payment method as")
  i = 0
  for adm in encoded_data['#pay']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print( str(i) + ". " + 'Unknown' )
  print('(Enter a number from 0 to ' + str(i) + ')')
  a9 = int(input())
  inp.append(a9)

  print( "Enter the water source category as")
  i = 0
  for adm in encoded_data['water_tech_category']:
    print( str(i) + ". " + str(adm) )
    i = i + 1
  print( str(i) + ". " + 'Unknown' )
  print('(Enter a number from 0 to ' + str(i) + ')')
  a10 = int(input())
  inp.append(a10)

  return inp

### Function for Prediction

In [ ]:
def make_prediction(input_data):

  column_names =['Classification as urban/semi-urban/rural_one-hot', 'log_Population',
        'water_source_category_one-hot', 'log_angle', '#status_id_one-hot',
        'log_days_passed', '#adm2_one-hot', 'log_dist', '#pay_one-hot',
        'water_tech_category_one-hot']

  data = input_data
  df = pd.DataFrame([data], columns = column_names)

  make_prediction = classifier.predict(df)
  print("The priority of target area is classified as: ")
  return  make_prediction  

## Gradio App

In [ ]:
def lagos(classification, population, water_source_category_area, latitude, longitude, status, days, local_government, payment, water_source_category):
  inp = []
  inp.append(classification)
  inp.append(lg(population))
  inp.append(water_source_category_area)
  latitude, longitude = cart2pol(latitude, longitude)
  inp.append(lg(latitude + 1000))
  inp.append(status)
  inp.append(lg(days))
  inp.append(local_government)
  inp.append(lg(longitude +1000))
  inp.append(payment)
  inp.append(water_source_category)
  prediction = make_prediction(inp)
  return prediction[0]

demo = gr.Interface(lagos, inputs=[gr.Dropdown([0, 1, 2], label= "Classification of Area (0. urban , 1. semi-urban , 2. rural)"),gr.Number(label= "Population of Area"),gr.Dropdown([0, 1], label= " Water Source Category of Area (0. well, 1. spring )"),
gr.Number(label= "Latitude of Area"),gr.Number(label= "Longitude of Area"),gr.Dropdown([0, 1], label= "Status Id of Water Source (0. Yes (Functional) , 1. No (non-functional)) "),
gr.Number(label= "Enter the number of days passed since installation of water source"),gr.Dropdown([0, 1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19], label= "Local Government (0. Kosofe , 1. Epe, 2. Ibeju-Lekki, 3. Alimosho , 4. Ikorodu, 5. Agege, 6. Ojo, 7. Badagry, 8. Lagos Mainland ,9. Mushin, 10. Ikeja, 11. Lagos Island, 12. Shomolu, 13. Eti-Osa, 14. Ifako-Ijaye, 15. Ajeromi-Ifelodun, 16. Surulere, 17. Oshodi, 18. Amuwo-Odofin)"),
gr.Dropdown([0, 1, 2, 3, 4], label= "Payment Method  (0. No, 1. Yes Point of collection, 2. Yes At breakdown, 3. Yes Periodic Levy, 4. Unknown)"),
gr.Dropdown([0, 1, 2, 3], label= "Water Source Category  (0. Mechanized Pump, 1. Tapstand, 2. Hand Pump, 3. Unknown)")], outputs=[gr.Textbox(label="Priority of Target Area is classified as")])
demo.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://59af978fb2cd9267.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


The priority of target area is classified as: 
Keyboard interruption in main thread... closing server.


(<gradio.routes.App at 0x7f492b2676d0>,
 'http://127.0.0.1:7860/',
 'https://59af978fb2cd9267.gradio.app')